In [1]:
import pandas as pd
import sys
import os 

# sys.path.append("../")
# sys.path.append("../../")
# sys.path.append("../../../")



In [5]:
from router.dev.shop.components.shop_product_card_list.main import load_brand_url

load_brand_url('consortium','a.p.c.')

'https://www.consortium.co.uk/brands/a-p-c.html'

In [3]:
from router.dev.utils.temp_file_manager import TempFileManager

await TempFileManager("platform_page").load_temp_file('product_detail')

({'kream_id': 15248,
  'kream_product_img_url': 'https://kream-phinf.pstatic.net/MjAyMTA4MjNfNjcg/MDAxNjI5NzA3NzQ5MjM1.8BSgjOBS89R2vmupO5G1h_XXAAYs4-DD3breNilvBfwg.JjPFmtmHC05SInImY2nkNkurF60i0TUtM17O1DO2SRUg.PNG/a_91f875b7e3bd4522a2dad91159818008.png?type=l',
  'kream_product_name': 'Adidas Samba OG Black White Gum',
  'brand_name': 'Adidas',
  'retail_price': '139,000원',
  'product_id': 'B75807',
  'product_release_date': '18/06/01',
  'color': 'core black/cloud white/gum',
  'wish': 84000,
  'review': 847,
  'updated_at': '2023-12-14 19:46:18'},)